In [36]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
import matplotlib.pyplot as plt
# Import stats from scipy library
from scipy import stats
import statsmodels.api as sm


from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector

# Including weather data

In [50]:
flights_fe = pd.read_csv(r"../data/interim/2_feature_engineered/train_data_Akhmet.csv")

In [51]:
flights_fe.head()

,mkt_unique_carrier,op_unique_carrier,op_carrier_fl_num,arr_delay,crs_elapsed_time,distance,inbound_flights,weather_data,arr_flights,dep_flights,...,hub_type_des,ownership_status_des,facility_use_des,area_des,fss_on_airport_des,beacon_color_des,total_operations_annual_des,Rank_des,origin_state,dest_state
0,DL,9E,5187,-17.0,125.0,692.0,4,Rain,0,0,...,Non-Hub,PUBLIC,PUBLIC,2278.0,NO,Clear and Green,87505,143.0,GA,PA
1,DL,9E,5443,-18.0,124.0,692.0,3,Fog,0,11,...,Non-Hub,PUBLIC,PUBLIC,2278.0,NO,Clear and Green,87505,143.0,GA,PA
2,DL,9E,5454,4.0,120.0,692.0,11,Precipitation,0,1,...,Non-Hub,PUBLIC,PUBLIC,2278.0,NO,Clear and Green,87505,143.0,GA,PA
3,DL,9E,5172,-15.0,123.0,692.0,7,Rain,0,4,...,Non-Hub,PUBLIC,PUBLIC,2278.0,NO,Clear and Green,87505,143.0,GA,PA
4,DL,9E,5172,-16.0,123.0,692.0,4,Fog,0,7,...,Non-Hub,PUBLIC,PUBLIC,2278.0,NO,Clear and Green,87505,143.0,GA,PA


In [52]:
df = flights_fe.copy()

In [53]:
X,y= df.drop(columns=["arr_delay"]),df[["arr_delay"]]

X_dummy = X.drop(columns=["op_carrier_fl_num",'beacon_color_or','beacon_color_des','ownership_status_des','facility_use_des','ownership_status_or','facility_use_or'])

In [54]:
X_dummy = pd.get_dummies(
    data=X_dummy,columns=["mkt_unique_carrier","op_unique_carrier",
                          'arr_month','arr_day_of_week','arr_hour','dep_month','dep_day_of_week','dep_hour', 
                          
                          'service_level_or','hub_type_or','fss_on_airport_or',
                          'service_level_des', 
                          'hub_type_des','fss_on_airport_des'  
                          
                          ,'origin_state','dest_state',
                          "weather_data"
                          ])

In [55]:
X_dummy.shape

(80043, 250)

In [56]:
X_train,X_test,y_train,y_test = train_test_split(X_dummy,y,train_size=0.7,random_state=0)

In [57]:
clf =LinearRegression()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [58]:
r2_score(y_test,y_pred)

0.061380590760721665

In [59]:
mean_absolute_error(y_test,y_pred)

12.373692696673572

In [60]:
mean_squared_error(y_test,y_pred)

246.77856860999717

In [61]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 0.3, n_estimators = 1000)
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=0.3, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0.300000012, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [62]:
y_pred = xg_reg.predict(X_test)
r2_score(y_test,y_pred)

0.06870818633297981

## Excluding weather data

In [63]:
X,y= df.drop(columns=["arr_delay"]),df[["arr_delay"]]

X_dummy = X.drop(columns=["op_carrier_fl_num",'beacon_color_or','beacon_color_des','ownership_status_des','facility_use_des','ownership_status_or','facility_use_or',"weather_data",])

In [64]:
X_dummy = pd.get_dummies(
    data=X_dummy,columns=["mkt_unique_carrier","op_unique_carrier",
                          'arr_month','arr_day_of_week','arr_hour','dep_month','dep_day_of_week','dep_hour', 
                          
                          'service_level_or','hub_type_or','fss_on_airport_or',
                          'service_level_des', 
                          'hub_type_des','fss_on_airport_des'  
                          
                          ,'origin_state','dest_state'
                          ])

In [65]:
X_dummy.shape

(80043, 243)

In [66]:
X_train,X_test,y_train,y_test = train_test_split(X_dummy,y,train_size=0.7,random_state=0)

In [67]:
clf =LinearRegression()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [68]:
r2_score(y_test,y_pred)

0.055185171851917625

In [69]:
mean_absolute_error(y_test,y_pred)

12.419744460079484

In [70]:
mean_squared_error(y_test,y_pred)

248.40744672098052

In [71]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 0.3, n_estimators = 1000)
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=0.3, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0.300000012, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [72]:
y_pred = xg_reg.predict(X_test)
r2_score(y_test,y_pred)

0.05949652070347089

In [83]:
X_test.shape

(24013, 243)

## Test data

In [73]:
test_data = pd.read_csv(r"../data/interim/2_feature_engineered/test_data.csv")

In [74]:
test = test_data.drop(columns=["op_carrier_fl_num",'beacon_color_or','beacon_color_des','ownership_status_des','facility_use_des','ownership_status_or','facility_use_or'])

In [75]:
drop_columns = list(set(test.columns)-set(X.columns))

In [76]:
X.shape

(80043, 35)

In [77]:
test.shape

(129720, 59)

In [80]:
test = test.drop(columns=drop_columns)

In [89]:
test.columns

Index(['mkt_unique_carrier', 'op_unique_carrier', 'crs_elapsed_time',
       'distance', 'inbound_flights', 'origin_state', 'dest_state',
       'service_level_or', 'hub_type_or', 'area_or', 'fss_on_airport_or',
       'total_operations_annual_or', 'Rank_or', 'service_level_des',
       'hub_type_des', 'area_des', 'fss_on_airport_des',
       'total_operations_annual_des', 'Rank_des', 'arr_month',
       'arr_day_of_week', 'arr_hour', 'dep_month', 'dep_day_of_week',
       'dep_hour'],
      dtype='object')

In [81]:
test_dummy = pd.get_dummies(
    data=test,columns=["mkt_unique_carrier","op_unique_carrier",
                          'arr_month','arr_day_of_week','arr_hour','dep_month','dep_day_of_week','dep_hour', 
                          
                          'service_level_or','hub_type_or','fss_on_airport_or',
                          'service_level_des', 
                          'hub_type_des','fss_on_airport_des'  
                          
                          ,'origin_state','dest_state'
                          ])

In [92]:
add_columns = set(X_dummy.columns)-set(test_dummy.columns)
for col in add_columns:
    test_dummy[col] = 0

In [97]:
drop_columns =list(set(test_dummy.columns)-set(X_dummy.columns))

In [99]:
test_dummy = test_dummy.drop(columns=drop_columns)

In [100]:
y_pred = xg_reg.predict(test_dummy)

In [101]:
y_pred

array([12.700654, 20.640152, 20.287792, ..., 29.56045 , 31.656145,
       17.507212], dtype=float32)

In [106]:
results = test_data[["fl_date","mkt_carrier","mkt_carrier_fl_num","origin","dest"]]

In [107]:
results["predicted delay"] = y_pred

D:\Anaconda\envs\geo_env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [109]:
results.to_csv(r"../reports/predictions.csv")